# Technical note: CVODE(S) simulation

Myokit's `Simulation` class uses the CVODE routine from the Sundials package to integrate systems of ODEs.

Given a system
\begin{align}
\dot{y}\left(t\right) &= f\left(y\left(t\right),u\left(t\right),t|p\right) \\
y(t_0) &= y_0
\end{align}

where $y$ is the state, $u$ are external inputs, and $p$ are parameters, CVODE can estimate $y(t)$ by integrating from $t_0$ to $t$.

In addition, Sundials provides a CVODES method that extends CVODE with the capability to perform _forward sensitivity analysis_, i.e. to calculate

\begin{equation}
\frac{\partial}{\partial p_i} y_j(t)$
\end{equation}
and
\begin{equation}
\frac{\partial}{\partial y_i(t=t_0)} y_j(t)$
\end{equation}

for any state $y_j$, parameter $p_i$, and initial condition $y_i(t=t_0)$.

Documentation for CVODE and CVODES can be found on the [Sundials website](https://computing.llnl.gov/projects/sundials)

## Naming

Starting from version 1.33.0, Myokit uses the CVODES C api for simulations with and without sensitivities.
However, as CVODES-without-sensitivities reduces to CVODE, we use both names throughout the documentation.

## Implementation choices

- Myokit uses CVODE's "Backwards Different Formulas" (BDF) integrator for stiff methods
- Because this is an implicit method, at each step a nonlinear system must be solved, for which the "Newton iteration" method is chosen.
- During integration, CVODE estimates the Jacobian $\frac{\partial f}{\partial y}$ using "difference quotients" (see the CVODE/CVODES manual section 2.1).

Step sizes:
- Instead of asking CVODES to integrate from some $t_\text{min}$ to $t_\text{max}$, Myokit uses the `CV_ONE_STEP` approach where CVODE just makes a single step of whatever length it determines is best.
- If this step passes over discontinuity (as indicated by the pacing protocol) at some time $t$, the state at $t$ is obtained (from CVODE, which uses a higher-order linear interpolation), and the simulation is reset to that point.
- If this step passes over one or multiple logging points, the state at those points is estimated (using interpolation again) without resetting the simulator.

## Sensitivities

todo

## CModel class

In Myokit version 1.33.0 and after, an attempt has been made to separate the CVODE(s) simulator code from the model code wherever possible.
This is described in detail in the `cmodel.h` header template, which implements a C model interface for Myokit models.